In this notebook, we shall use the stored PCA model and apply a kNN classifier in the training environment.

### 1. Check the attached training environment

Using '%attachments', we can see the training enviroment that is attached to the Notebook Cluster.

In [2]:
%attachments

Training Cluster    ML Engine
------------------  -----------
knntrainingenv      python


### 2. Write the training code

The training cluster that we have attached is 'knntrainingenv'. Using the '%%knntrainingenv', we build a kNN classification model in that training cluster. 

In [5]:
%%knntrainingenv
# We run the code directly into the training cluster with the help of the "%%" magic command

# Imports
import numpy as np
import pandas as pd
from sklearn import decomposition
import pickle
from sklearn.neighbors import KNeighborsClassifier

nfs_path = "/bd-fs-mnt/sj_ai_ml/"


# Load training data
train_label = np.load(nfs_path + "data/train_label.npy")
train_data = np.load(nfs_path + "data/train_data.npy")


# Load PCA transformation model that we dumped in the pre-processing step
with open(nfs_path + "models/pca_mnist.p", "rb") as f:
    pca_mnist = pickle.load(f)
    
pca_train = pca_mnist.transform(train_data)

# Define a k-NN model with 'n' neighbours. We use distance as weights which would 
# help make the closest digits gain more importance
def build_kNN_and_dump(neighbours, data, labels, save_path):
    kNN_model = KNeighborsClassifier(n_neighbors=neighbours, weights='distance')
    kNN_model.fit(data, labels)
    with open(save_path, "wb") as f:
        pickle.dump(kNN_model, f)
        
# Dump the model into the repository        
build_kNN_and_dump(3, pca_train, train_label, nfs_path + "models/kNN_mnist.p")

History URL: http://bluedata-42.bdlocal:10001/history/2


### 3. Examine the status

We examine the History URL returned from the earlier cell to see if the training job is finished. We use '%logs' to see this status

In [6]:
%logs --url http://bluedata-43.bdlocal:10001/history/2

Job Status: Finished






We now have successsfully dumped a k-NN classifier with 3 neighbours on the persistent storage. The next step would be to call this in the scoring script. We shall reigister this scoring script with a deployment and call it using a POST call.